In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re

In [19]:

def read_sparse_matrix(file):
    """
    reads in matrix via non-zero entry triples
    ex input:
    size: 3
    (0,0,123) (1,1,5) (2,1,-23)
    
    should output a matrix like:
    np.array([
        [123, 0,   0],
        [0,   5,   0],
        [0,   -23, 0]
    ])
    """
    m_size = int(file.readline().split()[1])
    m = np.zeros((m_size, m_size))
    for nnz_triple_str in file.readline().split():
        i, j, val = nnz_triple_str[1:-1].split(',')
        i = int(i)
        j = int(j)
        val = float(val)
        m[i,j] = val
    return m

def read_square_matrix(file):
    """
    function that reads in a square matrix from the file if the matrix is printed element by element separated by whitespace like:
    1 0 0
    2 1 0
    3 2 1
    """
    m = []
    cur_row = 1
    max_rows = 1    # placeholder value... since max_rows == # cols, we update this value in the loop
    while cur_row <= max_rows:
        line = file.readline()
        data = [float(x) for x in line.split()]
        m.append(data)
        if cur_row == 1:
            max_rows = len(data)    # number of rows = number of columns
        cur_row += 1
    return m


def read_data(f_path):
    data = {
        'Q': [],
        'policy_converged': [],
        # 'penalty_converged': []
    }
    with open(f_path) as f:
        while True:
            line = f.readline()
            if not line:
                return data
            if re.match("^Q", line):
                data['Q'].append(
                    # np.array(read_square_matrix(f))
                    read_sparse_matrix(f)
                )
            if re.match("^policy_converges", line):
                data['policy_converged'].append(
                    bool(int(f.readline()))
                )
            # if re.match("^penalty_converges", line):
            #     data['penalty_converged'].append(
            #         bool(int(f.readline()))
            #     )

In [20]:
data = read_data('./200.out')
data['eigenvalues'] = [np.linalg.eig(x) for x in data['Q']]
data['cond_num'] = [np.linalg.cond(x) for x in data['Q']]
data['Q_size'] = [x.shape[0] for x in data['Q']]

# angles...
# for Q in data['Q']:
#     for col in range(Q.shape[0]):
#         for row in range(Q.shape[0]):
#             if col == row:
#                 continue

data['Q_inv'] = [np.linalg.inv(x) for x in data['Q']]
data['Q_inv_min_val'] = [np.amin(x) for x in data['Q_inv']]
data['Q_inv_num_lt_0'] = [np.count_nonzero(x < 0) for x in data['Q_inv']]

In [21]:
pd_data = pd.DataFrame(data)

In [26]:
# pd_data[pd_data.policy_converged == False]
pd_data[pd_data.Q_inv_min_val < 0]

,Q,policy_converged,eigenvalues,cond_num,Q_size,Q_inv,Q_inv_min_val,Q_inv_num_lt_0
1,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([6.907769999999995, 6.907770000000004, 6.9077...",1.000000,28,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-3.722676e-17,8
3,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([6.90777, 6.907769999999993, 6.90776999999999...",1.000000,28,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-1.675202e-16,6
4,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([6.907769999999983, 6.907770000000015, 6.9077...",1.000000,28,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-2.931600e-16,6
5,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([(6.907769999999977+0j), (6.907770000000019+0...",1.000000,56,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-4.094931e-16,32
6,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([3.4538800000000003, 6.907770000000001, 6.907...",2.000003,84,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-2.698938e-16,46
7,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([6.90777000000001, 6.90776999999999, 6.907770...",1.000000,56,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-3.396945e-16,32
8,"[[6.90777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,"([3.453880000000001, 6.907770000000003, 3.4538...",2.000003,84,"[[0.14476451879550128, 0.0, 0.0, 0.0, 0.0, 0.0...",-2.792007e-16,46
